In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install pytorch-pretrained-bert

In [21]:
# 토큰화 같은 전처리 과정을 담고 있는 코드
# 텍스트 데이터 path 수정해야 함.

import os
import csv
import logging
import random
import sys
import numpy as np
import requests
import torch
from random import shuffle
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

from pathlib import Path
from urllib.parse import urlparse

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)

            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines

class PgProcessor(DataProcessor):

    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "/content/gdrive/MyDrive/BOAZ/mini_pj/train.tsv")), "train") # 올바른 파일 경로 입력

    def get_test_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "/test.tsv")), "test") # 올바른 파일 경로 입력

    def get_labels(self):
        return ["0", "1", "2", "3", "4", "5", "6"]

    def _create_examples(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[0]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        # label_id = label_map[example.label]
        label_id = example.label
        label_id = float(label_id)
        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""
    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop(0) #For dialogue context
        else:
            tokens_b.pop()

In [22]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install torchmetrics

  Using cached sentencepiece-0.1.91.tar.gz (500 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached transformers-4.8.2-py3-none-any.whl.metadata (48 kB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl.metadata (5.6 kB)
  Using cached sacremoses-0.1.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.8.2-py3-none-any.whl (2.5 MB)
Using cached 

In [23]:
from __future__ import absolute_import, division, print_function
import argparse
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from kobert_tokenizer import KoBERTTokenizer
import os
import random
import logging

# Set up logging
logger = logging.getLogger(__name__)

In [24]:
def get_args():
    class Args:
        def __init__(self):
            self.data_dir = '/content/gdrive/MyDrive/BOAZ/mini_pj/'
            self.bert_model = 'skt/kobert-base-v1'
            self.task_name = 'Multi'
            self.output_dir = '/content/gdrive/MyDrive/BOAZ/mini_pj/CM-BERT_output/'
            self.cache_dir = ""
            self.max_seq_length = 100
            self.do_train = True
            self.do_test = True
            self.do_lower_case = True
            self.train_batch_size = 24
            self.eval_batch_size = 24
            self.test_batch_size = 24
            self.learning_rate = 2e-5
            self.num_train_epochs = 3.0
            self.warmup_proportion = 0.1
            self.no_cuda = False
            self.seed = 11111
            self.gradient_accumulation_steps = 1
    return Args()

args = get_args()

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

def load_audio_data(batch_dir, num_batches):
    audio_data = []
    for i in range(num_batches):
        batch_path = f'{batch_dir}/batch_{i}.npy'
        batch_data = np.load(batch_path)
        audio_data.append(batch_data)
    return np.concatenate(audio_data, axis=0)

# Define batch directory and number of batches
batch_dir = '/content/gdrive/MyDrive/BOAZ/mini_pj/all_mfcc_npy'
num_batches = 39

# Load audio data
audio_npy = load_audio_data(batch_dir, num_batches)
train_audio, test_audio = audio_npy[:15499], audio_npy[15499:]


In [5]:
# InputExample 클래스 정의
class InputExample(object):
    def __init__(self, guid, text_a=None, text_b=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

# InputFeatures 클래스 정의
class InputFeatures(object):
    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

# convert_examples_to_features 함수 수정
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)
        tokens_b = tokenizer.tokenize(example.text_b) if example.text_b else []

        if len(tokens_a) + len(tokens_b) > max_seq_length - 3:
            tokens_a = tokens_a[:(max_seq_length - len(tokens_b) - 3)]

        tokens = ["[CLS]"] + tokens_a + ["[SEP]"] + tokens_b + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        segment_ids = [0] * (len(tokens_a) + 2) + [1] * (len(tokens_b) + 1)

        padding_length = max_seq_length - len(input_ids)
        input_ids += [0] * padding_length
        input_mask += [0] * padding_length
        segment_ids += [0] * padding_length

        label_id = label_map.get(example.label, 0)  # Ensure label_id is properly mapped

        features.append(
            InputFeatures(
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                label_id=label_id
            )
        )

    return features

In [25]:
# 데이터 로딩 함수
def load_data(args):
    # Define paths to your data files
    train_data_path = os.path.join(args.data_dir, "/content/gdrive/MyDrive/BOAZ/mini_pj/train.tsv")
    test_data_path = os.path.join(args.data_dir, "/content/gdrive/MyDrive/BOAZ/mini_pj/test.tsv")

    if not os.path.exists(train_data_path) or not os.path.exists(test_data_path):
        raise FileNotFoundError("Train or test data file not found.")

    # Load train and test data
    train_df = pd.read_csv(train_data_path, delimiter='\t')
    test_df = pd.read_csv(test_data_path, delimiter='\t')

    return train_df, test_df

In [26]:
from transformers import BertTokenizer, BertForSequenceClassification
import os

def setup_model_and_data(args):
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    n_gpu = torch.cuda.device_count()
    print(f"device: {device} n_gpu: {n_gpu}")

    if args.gradient_accumulation_steps < 1:
        raise ValueError(f"Invalid gradient_accumulation_steps parameter: {args.gradient_accumulation_steps}, should be >= 1")

    args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps

    seed_num = np.random.randint(1, 10000)
    random.seed(seed_num)
    np.random.seed(seed_num)
    torch.manual_seed(seed_num)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed_num)

    if not args.do_train and not args.do_test:
        raise ValueError("At least one of `do_train` or `do_test` must be True.")

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    task_name = args.task_name.lower()

    processors = {
        "multi": PgProcessor,
    }

    num_labels_task = {
        "multi": 7,
    }

    if task_name not in processors:
        raise ValueError(f"Task not found: {task_name}")

    processor = processors[task_name]()
    num_labels = num_labels_task[task_name]
    label_list = processor.get_labels()

    tokenizer = KoBERTTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case)

    # Define cache_dir explicitly
    cache_dir = args.cache_dir if args.cache_dir else './cache'

    model = BertForSequenceClassification.from_pretrained(args.bert_model, cache_dir=cache_dir, num_labels=num_labels)

    for name, param in model.named_parameters():
        param.requires_grad = False
        if "encoder.layer.0" in name or "encoder.layer.1" in name:
            param.requires_grad = True
        if "encoder.layer.2" in name or "encoder.layer.3" in name:
            param.requires_grad = True
        if "encoder.layer.4" in name or "encoder.layer.5" in name:
            param.requires_grad = True
        if "encoder.layer.6" in name or "encoder.layer.7" in name:
            param.requires_grad = True
        if "encoder.layer.8" in name or "encoder.layer.9" in name:
            param.requires_grad = True
        if "encoder.layer.10" in name or "encoder.layer.11" in name:
            param.requires_grad = True
        if "BertFinetun" in name or "pooler" in name:
            param.requires_grad = True

    model.to(device)

    if n_gpu > 1:
        model = torch.nn.DataParallel(model)

    return model, tokenizer, processor, num_labels, label_list, device
# 모델 학습 함수
def train_model(args, model, tokenizer, processor, device, train_audio):
    if args.do_train:
        train_examples = processor.get_train_examples(args.data_dir)
        num_train_optimization_steps = int(len(train_examples) / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs

        train_features = convert_examples_to_features(train_examples, label_list, args.max_seq_length, tokenizer)
        print("***** Running training *****")
        print("  Num examples = %d", len(train_examples))
        print("  Batch size = %d", args.train_batch_size)
        print("  Num steps = %d", num_train_optimization_steps)

        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.float32)
        all_train_audio = torch.tensor(train_audio, dtype=torch.float32)

        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_train_audio, all_label_ids)
        train_sampler = RandomSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)

        # Define no_decay and new_decay parameters
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        new_decay = ['BertFine']

        param_optimizer = list(model.named_parameters())

        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and not any(np in n for np in new_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and any(np in n for np in new_decay)], 'lr': 0.01}
        ]

        # Use AdamW optimizer from transformers library
        optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, correct_bias=False)

        global_step = 0
        tr_loss = 0
        model.train()

        for _ in trange(int(args.num_train_epochs), desc="Epoch"):
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, train_audio, label_ids = batch
                model.zero_grad()
                loss = model(input_ids, train_audio, segment_ids, input_mask, labels=label_ids)

                if n_gpu > 1:
                    loss = loss.mean()  # mean() to average on multi-g

                if args.gradient_accumulation_steps > 1:
                    loss = loss / args.gradient_accumulation_steps

                loss.backward()

                tr_loss += loss.item()
                if (step + 1) % args.gradient_accumulation_steps == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    global_step += 1

        return model



In [27]:
try:
    # 모델 및 데이터 설정 함수 호출
    model, tokenizer, processor, num_labels, label_list, device, train_audio = setup_model_and_data(args)
    print("Model and data setup successful!")
except Exception as e:
    # 오류가 발생했을 때 오류 메시지 출력
    print(f"Error in setup_model_and_data: {e}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


device: cpu n_gpu: 0


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error in setup_model_and_data: not enough values to unpack (expected 7, got 6)


In [10]:
# 모델 학습 실행
try:
    trained_model = train_model(args, model, tokenizer, processor, device)
    print("Model training successful!")
except Exception as e:
    print(f"Error during training: {e}")

Error during training: name 'model' is not defined


In [41]:
def evaluate_model(model, tokenizer, processor, device, eval_data_dir):
    model.eval()
    eval_examples = processor.get_dev_examples(eval_data_dir)
    eval_features = convert_examples_to_features(eval_examples, label_list, args.max_seq_length, tokenizer)

    all_input_ids = torch.tensor([f['input_ids'] for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f['input_mask'] for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f['segment_ids'] for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f['label_id'] for f in eval_features], dtype=torch.float32)

    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    eval_dataloader = DataLoader(eval_data, batch_size=args.eval_batch_size)

    preds = []
    labels = []
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        with torch.no_grad():
            outputs = model(input_ids, segment_ids, input_mask)
            logits = outputs[0]
        preds.append(logits.cpu().numpy())
        labels.append(label_ids.cpu().numpy())

    preds = np.concatenate(preds, axis=0)
    labels = np.concatenate(labels, axis=0)

    # Evaluation metrics
    accuracy = accuracy_score(labels, np.argmax(preds, axis=1))
    f1 = f1_score(labels, np.argmax(preds, axis=1), average='weighted')

    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")

# 모델 평가 실행
evaluate_model(trained_model, tokenizer, processor, device, args.data_dir)


NameError: name 'trained_model' is not defined